# GDATABT
## Ahmet Emre Usta
## Engincan Göçer
### Hacettepe University
### 25.04.2023

In [1]:
import pandas as pd

In [2]:
xls = pd.ExcelFile('Data23.xlsx')
xls.sheet_names

['SALES', 'PRODUCTS', 'CUSTOMERS']

In [3]:
df_sales = pd.read_excel(xls, "SALES")
df_products = pd.read_excel(xls, "PRODUCTS")
df_customers = pd.read_excel(xls, "CUSTOMERS")

In [4]:
df_sales["DateTime"] = pd.to_datetime(df_sales["DateTime"])
df_customers["UserFirstTransaction"] = pd.to_datetime(df_customers["UserFirstTransaction"])

In [5]:
# split the sales data into two dataframes based on the moth of the sales
df_sales_1 = df_sales[df_sales["DateTime"] <= "2017-10-01"].reset_index(drop=True)
df_sales_2 = df_sales[df_sales["DateTime"] > "2017-10-01"].reset_index(drop=True)

In [6]:
first_6_months = df_sales_2[df_sales_2["DateTime"] <= "2018-04-01"]
last_3_months = df_sales_2[df_sales_2["DateTime"] > "2018-04-01"]

In [7]:
churned_users = []
active_users = []

for user in df_customers["UserID"]:
    if user in first_6_months["UserID"].unique() and user in last_3_months["UserID"].unique():
        active_users.append(user)
        # print(f"User {user} is active")
    if user in first_6_months["UserID"].unique() and user not in last_3_months["UserID"].unique():
        churned_users.append(user)
        # print(f"User {user} is chruned")

In [8]:
#take columns from df_customers which is same as the first_6_months["UserID"] and create a new dataframe
df_train = df_customers[df_customers["UserID"].isin(first_6_months["UserID"])].reset_index(drop=True)
df_train.head()

,UserID,UserFirstTransaction,Gender,Location,Age
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31


In [9]:
#add churn column to the df_train dataframe
df_train["isChurn"] = df_train["UserID"].apply(lambda x: 1 if x in churned_users else 0)

In [10]:
df_train.head()

,UserID,UserFirstTransaction,Gender,Location,Age,isChurn
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19,1
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34,1
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39,0
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42,0
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31,0


In [11]:
#add total price column to the df_train dataframe
df_train["TotalPrice"] = df_train["UserID"].apply(lambda x: first_6_months[first_6_months["UserID"] == x]["Price"].sum())

In [12]:
df_train.head()

,UserID,UserFirstTransaction,Gender,Location,Age,isChurn,TotalPrice
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19,1,146.70
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34,1,443.55
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39,0,917.55
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42,0,100.50
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31,0,1773.90


In [13]:
df_train["TotalSales"] = df_train["UserID"].apply(lambda x: first_6_months[first_6_months["UserID"] == x]["Price"].count())

In [14]:
df_train.head()

,UserID,UserFirstTransaction,Gender,Location,Age,isChurn,TotalPrice,TotalSales
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19,1,146.70,1
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34,1,443.55,8
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39,0,917.55,7
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42,0,100.50,2
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31,0,1773.90,6


In [15]:
first_6_months.loc[:, 'Discount'] = first_6_months['Discount'].replace({'Yes': 1, 'No': 0}).astype(int)
df_train["TotalDiscount"] = df_train["UserID"].apply(lambda x: first_6_months[first_6_months["UserID"] == x]["Discount"].sum())

/var/folders/hs/7w96q1ln53d4q68gvt_vn2wr0000gn/T/ipykernel_89757/1011940479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_6_months.loc[:, 'Discount'] = first_6_months['Discount'].replace({'Yes': 1, 'No': 0}).astype(int)
/var/folders/hs/7w96q1ln53d4q68gvt_vn2wr0000gn/T/ipykernel_89757/1011940479.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  first_6_months.loc[:, 'Discount'] = first_6_months['Discount'].replace({'Yes': 1, 'No': 0}).astype(int)


In [16]:
# fid user last transaction date and add it to the df_train dataframe
df_train["UserLastTransaction"] = df_train["UserID"].apply(lambda x: first_6_months[first_6_months["UserID"] == x]["DateTime"].max())

In [17]:
df_train

,UserID,UserFirstTransaction,Gender,Location,Age,isChurn,TotalPrice,TotalSales,TotalDiscount,UserLastTransaction
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19,1,146.70,1,0,2017-12-17 18:08:28.150
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34,1,443.55,8,0,2018-02-11 19:39:53.710
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39,0,917.55,7,0,2018-03-17 18:41:21.750
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42,0,100.50,2,0,2017-10-23 18:51:56.450
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31,0,1773.90,6,0,2018-03-28 18:47:27.690
...,...,...,...,...,...,...,...,...,...,...
2147,500611425,2004-05-20 00:00:00.000,FEMALE,KAYSERI,42,0,1492.86,10,1,2018-03-20 19:31:17.820
2148,500611491,2015-10-29 00:00:00.000,MALE,ANTALYA,27,0,968.49,17,0,2018-03-13 20:01:58.300
2149,500611887,2017-12-31 14:05:02.080,MALE,IZMIR,48,0,443.46,6,0,2018-03-25 12:35:52.720
2150,500612481,2017-08-01 13:00:19.640,MALE,ANTALYA,35,1,306.00,4,0,2017-12-30 13:59:08.520


In [18]:
# find user active days from UserLastTransaction - UserFirstTransaction and add it to the df_train dataframe
df_train["UserActiveDays"] = df_train["UserID"].apply(lambda x: (df_train[df_train["UserID"] == x]["UserLastTransaction"] - df_train[df_train["UserID"] == x]["UserFirstTransaction"]).dt.days.values[0])

In [19]:
first_6_months.head()

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount
0,29551,500522226,2017-10-01 00:09:28.360,10224,MOBILE,Cash,60.0,0
1,29552,500301456,2017-10-01 00:14:02.880,10224,MOBILE,Cash,57.0,0
2,29553,500279071,2017-10-01 01:23:00.270,10481,MOBILE,Cash,45.0,0
3,29554,500387630,2017-10-01 01:23:12.890,10127,WEB,Cash,90.0,0
4,29555,500370316,2017-10-01 02:50:03.780,10720,WEB,Cash,31.5,0


In [20]:
first_6_months.loc[:, 'Channel'] = first_6_months['Channel'].replace({'WEB': 1, 'MOBILE': 0}).astype(int)
first_6_months["Channel"].unique()

/var/folders/hs/7w96q1ln53d4q68gvt_vn2wr0000gn/T/ipykernel_89757/555804511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_6_months.loc[:, 'Channel'] = first_6_months['Channel'].replace({'WEB': 1, 'MOBILE': 0}).astype(int)
/var/folders/hs/7w96q1ln53d4q68gvt_vn2wr0000gn/T/ipykernel_89757/555804511.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  first_6_months.loc[:, 'Channel'] = first_6_months['Channel'].replace({'WEB': 1, 'MOBILE': 0}).astype(int)


array([0, 1])

In [21]:
# find user total web channel and add it to the df_train dataframe
df_train["TotalWeb"] = df_train["UserID"].apply(lambda x: first_6_months[(first_6_months["UserID"] == x) & (first_6_months["Channel"] == 1)]["Channel"].count())

df_train["TotalMobile"] = df_train["UserID"].apply(lambda x: first_6_months[(first_6_months["UserID"] == x) & (first_6_months["Channel"] == 0)]["Channel"].count())

In [22]:
first_6_months["PaymentType"].unique()

array(['Cash', 'Online Credit Card', 'Mobile Payment'], dtype=object)

In [23]:
first_6_months.loc[:, 'PaymentType'] = first_6_months['PaymentType'].replace({'Cash': 1, 'Mobile Payment': 2, 'Online Credit Card': 3}).astype(int)
first_6_months["PaymentType"].unique()

/var/folders/hs/7w96q1ln53d4q68gvt_vn2wr0000gn/T/ipykernel_89757/34956761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_6_months.loc[:, 'PaymentType'] = first_6_months['PaymentType'].replace({'Cash': 1, 'Mobile Payment': 2, 'Online Credit Card': 3}).astype(int)
/var/folders/hs/7w96q1ln53d4q68gvt_vn2wr0000gn/T/ipykernel_89757/34956761.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  first_6_months.loc[:, 'PaymentType'] = first_6_months['PaymentType'].replace({'Cash': 1, 'Mobile Payment': 2, 'Online Credit Card

array([1, 3, 2])

In [24]:
# find user total web channel and add it to the df_train dataframe
df_train["TotalCashPayment"] = df_train["UserID"].apply(lambda x: first_6_months[(first_6_months["UserID"] == x) & (first_6_months["PaymentType"] == 1)]["PaymentType"].count())

df_train["TotalMobilPayment"] = df_train["UserID"].apply(lambda x: first_6_months[(first_6_months["UserID"] == x) & (first_6_months["PaymentType"] == 2)]["PaymentType"].count())

df_train["TotalOnlinePayment"] = df_train["UserID"].apply(lambda x: first_6_months[(first_6_months["UserID"] == x) & (first_6_months["PaymentType"] == 3)]["PaymentType"].count())

In [25]:
# find total days from today to user last transaction and add it to the df_train dataframe
df_train["UserLastTransactionDays"] = df_train["UserID"].apply(lambda x: (pd.to_datetime("today") - df_train[df_train["UserID"] == x]["UserLastTransaction"]).dt.days.values[0])

In [26]:
tmp_df = first_6_months[["UserID","ProductID"]]

merged_df = pd.merge(tmp_df, df_products, on='ProductID', how='left')

category_pivot = pd.pivot_table(merged_df, values='ProductID', index='UserID', columns='Category', aggfunc='count', fill_value=0)


In [27]:
# merge the category_pivot dataframe with the df_train dataframe
df_train = pd.merge(df_train, category_pivot, on='UserID', how='left')
df_train.head()

,UserID,UserFirstTransaction,Gender,Location,Age,isChurn,TotalPrice,TotalSales,TotalDiscount,UserLastTransaction,...,Hobbies,Kitchen Electronics,Male Fashion,Male Shoes,Outdoor Sports,Smart Phones,Smart Watches,Sound Systems,Sport Shoes,TVs and TV Sets
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19,1,146.70,1,0,2017-12-17 18:08:28.150,...,0,0,0,0,0,0,0,0,0,1
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34,1,443.55,8,0,2018-02-11 19:39:53.710,...,0,0,1,4,0,2,0,0,0,1
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39,0,917.55,7,0,2018-03-17 18:41:21.750,...,0,0,0,0,0,0,0,0,1,1
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42,0,100.50,2,0,2017-10-23 18:51:56.450,...,0,0,0,0,0,1,0,0,0,0
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31,0,1773.90,6,0,2018-03-28 18:47:27.690,...,0,0,3,2,0,0,0,0,0,0


In [28]:
#We need to add "Indoor Sports" column to make the

In [29]:
hobbies_loc = df_train.columns.get_loc('Hobbies')

In [30]:
hobbies_loc

21

In [31]:
df_train.insert(loc=22, column='Indoor Sports', value=0)

In [32]:
df_train

,UserID,UserFirstTransaction,Gender,Location,Age,isChurn,TotalPrice,TotalSales,TotalDiscount,UserLastTransaction,...,Indoor Sports,Kitchen Electronics,Male Fashion,Male Shoes,Outdoor Sports,Smart Phones,Smart Watches,Sound Systems,Sport Shoes,TVs and TV Sets
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19,1,146.70,1,0,2017-12-17 18:08:28.150,...,0,0,0,0,0,0,0,0,0,1
1,500234752,2013-10-24 00:00:00.000,MALE,ISTANBUL,34,1,443.55,8,0,2018-02-11 19:39:53.710,...,0,0,1,4,0,2,0,0,0,1
2,500234763,2017-08-20 20:18:42.910,MALE,ADANA,39,0,917.55,7,0,2018-03-17 18:41:21.750,...,0,0,0,0,0,0,0,0,1,1
3,500234829,2009-09-01 00:00:00.000,MALE,ESKISEHIR,42,0,100.50,2,0,2017-10-23 18:51:56.450,...,0,0,0,0,0,1,0,0,0,0
4,500234895,2005-11-10 00:00:00.000,MALE,ISTANBUL,31,0,1773.90,6,0,2018-03-28 18:47:27.690,...,0,0,3,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,500611425,2004-05-20 00:00:00.000,FEMALE,KAYSERI,42,0,1492.86,10,1,2018-03-20 19:31:17.820,...,0,0,0,1,0,2,0,1,1,1
2148,500611491,2015-10-29 00:00:00.000,MALE,ANTALYA,27,0,968.49,17,0,2018-03-13 20:01:58.300,...,0,0,0,0,0,1,0,3,0,3
2149,500611887,2017-12-31 14:05:02.080,MALE,IZMIR,48,0,443.46,6,0,2018-03-25 12:35:52.720,...,0,0,0,0,0,0,0,0,0,3
2150,500612481,2017-08-01 13:00:19.640,MALE,ANTALYA,35,1,306.00,4,0,2017-12-30 13:59:08.520,...,0,0,0,0,0,0,0,0,0,4


In [33]:
df_train.columns

Index(['UserID', 'UserFirstTransaction', 'Gender', 'Location', 'Age',
       'isChurn', 'TotalPrice', 'TotalSales', 'TotalDiscount',
       'UserLastTransaction', 'UserActiveDays', 'TotalWeb', 'TotalMobile',
       'TotalCashPayment', 'TotalMobilPayment', 'TotalOnlinePayment',
       'UserLastTransactionDays', 'Computers & Laptops',
       'Electronic Accessories', 'Female Fashion', 'Female Shoes', 'Hobbies',
       'Indoor Sports', 'Kitchen Electronics', 'Male Fashion', 'Male Shoes',
       'Outdoor Sports', 'Smart Phones', 'Smart Watches', 'Sound Systems',
       'Sport Shoes', 'TVs and TV Sets'],
      dtype='object')

In [34]:
df_test = df_train

In [35]:
df_test.to_csv("test.csv", index = False)